In [48]:
# Have a conversational flow to manage memory
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read the local .env file
openai.api_key = os.environ['OPENAI_API_KEY'] # this is now the API key to use

testEnvString = os.environ['TEST_ENV'] # test to see if .env is working
print(testEnvString)

hello there!


In [49]:
# Here, we're having LangChain try and mimic a chatbot conversation
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
llm_model = "gpt-4o"

In [50]:


llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True # verbose will give you a bit more context into what the system is thinking. if you just want the output itself, verbose=False
)

In [51]:
# Let's build a CONVERSATION CHAIN! Chain will come later
conversation.predict(input="Hello, can you help me generate test problems?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, can you help me generate test problems?
AI:

> Finished chain.


"Hello! I'd be happy to help you generate test problems. What subject or topic are you focusing on? For example, are you looking for math problems, science questions, language exercises, or something else? Let me know so I can tailor the problems to your needs!"

In [52]:
conversation.predict(input="What's 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, can you help me generate test problems?
AI: Hello! I'd be happy to help you generate test problems. What subject or topic are you focusing on? For example, are you looking for math problems, science questions, language exercises, or something else? Let me know so I can tailor the problems to your needs!
Human: What's 1+1?
AI:

> Finished chain.


"1+1 is 2! That's a classic example of a basic arithmetic problem. If you're looking for more math problems, I can certainly help with that. Would you like problems in addition, subtraction, multiplication, division, or perhaps something more advanced like algebra or geometry? Let me know!"

In [53]:
conversation.predict(input="What did I ask you to generate earlier?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, can you help me generate test problems?
AI: Hello! I'd be happy to help you generate test problems. What subject or topic are you focusing on? For example, are you looking for math problems, science questions, language exercises, or something else? Let me know so I can tailor the problems to your needs!
Human: What's 1+1?
AI: 1+1 is 2! That's a classic example of a basic arithmetic problem. If you're looking for more math problems, I can certainly help with that. Would you like problems in addition, subtraction, multiplication, division, or perhaps something more advanced like algebra or geometry? Let me know!
Human: What did I ask you to generate earlier

'You asked me to help you generate test problems. If you have a specific subject or type of problem in mind, please let me know, and I can create some tailored examples for you!'

In [54]:
print(memory.buffer) # this allows us to store the entire conversation, so far

Human: Hello, can you help me generate test problems?
AI: Hello! I'd be happy to help you generate test problems. What subject or topic are you focusing on? For example, are you looking for math problems, science questions, language exercises, or something else? Let me know so I can tailor the problems to your needs!
Human: What's 1+1?
AI: 1+1 is 2! That's a classic example of a basic arithmetic problem. If you're looking for more math problems, I can certainly help with that. Would you like problems in addition, subtraction, multiplication, division, or perhaps something more advanced like algebra or geometry? Let me know!
Human: What did I ask you to generate earlier?
AI: You asked me to help you generate test problems. If you have a specific subject or type of problem in mind, please let me know, and I can create some tailored examples for you!


In [55]:
memory.load_memory_variables({}) # Returns the actual object to see the entire conversation so far

{'history': "Human: Hello, can you help me generate test problems?\nAI: Hello! I'd be happy to help you generate test problems. What subject or topic are you focusing on? For example, are you looking for math problems, science questions, language exercises, or something else? Let me know so I can tailor the problems to your needs!\nHuman: What's 1+1?\nAI: 1+1 is 2! That's a classic example of a basic arithmetic problem. If you're looking for more math problems, I can certainly help with that. Would you like problems in addition, subtraction, multiplication, division, or perhaps something more advanced like algebra or geometry? Let me know!\nHuman: What did I ask you to generate earlier?\nAI: You asked me to help you generate test problems. If you have a specific subject or type of problem in mind, please let me know, and I can create some tailored examples for you!"}

In [56]:
# You can also set memory
memoryNew = ConversationBufferMemory()
memoryNew.save_context({"input": "Hi"}, {"output": "Wazzzzzapppppp"})

memoryNew.load_memory_variables({})

{'history': 'Human: Hi\nAI: Wazzzzzapppppp'}

In [57]:
# To add more memory, you can keep on adding and appending NEW stuff in it with save_context
# For LLM chat conversation, the LLM is stateless and it doesn't remember anything

# Usually, wrapper code is provided so the memory can provide context to each INDEPENDENT call. this is usually considered a preamble

# LangChain provides good way to store different kinds of memories

# BUFFERWINDOW memory: it only remembers the last x conversations (only keeps a window of memories)

from langchain.memory import ConversationBufferWindowMemory
# ^ with this, you can do this to prevent the memory going way too long
# this bufferwindow memory can help save TOKENS (save money!)

from langchain.memory import ConversationTokenBufferMemory # we can also use this one, to include a max token limit to save on cost and keep cost consistent

# we can also have memory to SUMMARIZE the current memory conversation for us!
from langchain.memory import ConversationSummaryBufferMemory

# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

memory.load_memory_variables({})


conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

conversation.predict(input="What would be a good demo to show?")

memory.load_memory_variables({})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and mentions they are just hanging out. The AI responds casually, and the human asks what is on the schedule for today.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


{'history': "System: The human greets the AI and mentions they are just hanging out. The AI responds casually, and the human asks what is on the schedule for today. The AI outlines a busy schedule including a meeting at 8am, time to work on a LangChain project, and a lunch meeting with a customer to discuss the latest in AI. The human asks for a good demo to show, and the AI suggests several ideas including text generation, question answering, summarization, translation, sentiment analysis, code generation, and an interactive chatbot. The AI also advises tailoring the demo to the customer's specific interests and ensuring all necessary preparations are made for a smooth presentation."}

In [ ]:
# Additional memory types
# Vector data memory: Stores text (from conversation or elsewhere) in a vector database, retrieve most relevant blocks of text

# Entity memories
# Using LLM, remember details about specific entities.
# This is when you want it to remember details and facts about a certain person, etc.

# You can use several different kinds of memory here in LangChain as well. You can also just store the whole conversation here too.